In [44]:
import sys
sys.path.append('../')
from datasets import load_dataset
from web.model import load_model
dataset = load_dataset("taidng/UIT-ViQuAD2.0")

In [45]:
test_df= dataset['validation']

In [46]:
test_df = test_df.to_pandas()
test_df.head()

,id,uit_id,title,context,question,answers,is_impossible,plausible_answers
0,0001-0001-0001,uit_000001,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Paris đạt được thành quả gì sau khoảng 4 thế k...,{'text': ['trở thành một trong những trung tâm...,False,None
1,0001-0001-0002,uit_000002,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Vị trí địa lý của Pháp có gì đặc biệt?,"{'text': [], 'answer_start': []}",True,{'text': ['nằm ở điểm gặp nhau của các hành tr...
2,0001-0001-0003,uit_000003,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Kinh tế xung quanh kinh đô ánh sáng mạnh về gì?,"{'text': ['giáo dục và nghệ thuật', 'nông nghi...",False,None
3,0001-0001-0004,uit_000004,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Cuộc cách mạng Pháp diễn ra ở đâu?,"{'text': ['Paris'], 'answer_start': [358]}",False,None
4,0001-0001-0005,uit_000005,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Kinh đô ánh sáng nổi tiếng về lĩnh vực gì?,"{'text': ['nghệ thuật và giải trí', 'trung tâm...",False,None


In [47]:
df = test_df[["id", "question", "answers", "context"]]

In [48]:
df['answer'] = df['answers'].apply(lambda x: x['text'])
df['answer_start'] = df['answers'].apply(lambda x: x['answer_start'])

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1744\763388355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = df['answers'].apply(lambda x: x['text'])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1744\763388355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer_start'] = df['answers'].apply(lambda x: x['answer_start'])


In [49]:
df= df[['id', 'question', 'context', 'answer', 'answer_start']]

In [50]:
# Convert list to string
df['answer'] = df['answer'].apply(lambda x: ', '.join(map(str, x)) if len(x) > 0 else '')
# Convert list to string
df['answer_start'] = df['answer_start'].apply(lambda x: ', '.join(map(str, x)) if len(x) > 0 else '')
# Remove rows with empty 'answer'
df = df[df['answer'].str.strip() != '']

In [51]:
df.head()

,id,question,context,answer,answer_start
0,0001-0001-0001,Paris đạt được thành quả gì sau khoảng 4 thế k...,Paris nằm ở điểm gặp nhau của các hành trình t...,trở thành một trong những trung tâm văn hóa củ...,565
2,0001-0001-0003,Kinh tế xung quanh kinh đô ánh sáng mạnh về gì?,Paris nằm ở điểm gặp nhau của các hành trình t...,"giáo dục và nghệ thuật, nông nghiệp","323, 109"
3,0001-0001-0004,Cuộc cách mạng Pháp diễn ra ở đâu?,Paris nằm ở điểm gặp nhau của các hành trình t...,Paris,358
4,0001-0001-0005,Kinh đô ánh sáng nổi tiếng về lĩnh vực gì?,Paris nằm ở điểm gặp nhau của các hành trình t...,"nghệ thuật và giải trí, trung tâm văn hóa của ...","634, 591"
5,0001-0001-0006,Paris là một thành phố quan trọng của nước Phá...,Paris nằm ở điểm gặp nhau của các hành trình t...,thế kỷ 10,134


### Evaluate Model

In [52]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = load_model('../model/qa_large_model/')

In [61]:
question = list(df['question'])
context = list(df['context'])
answer_actual = list(df['answer'])

In [62]:
import torch
from sklearn.metrics import accuracy_score, f1_score

def run_prediction(model, tokenizer, context, question, answer_actual):
    inputs = tokenizer.encode_plus(
        question, context, 
        return_tensors='pt'
    ).to('cpu')
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs[0])  
    answer_end = torch.argmax(outputs[1]) + 1 

    predicted_answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(
            inputs.input_ids[0][answer_start:answer_end]
        )
    )
    
    print(
        f'- Question: {question}\n'
        f'-Context: {context}\n'
        f'- Predicted answer: {predicted_answer}\n'
        f'- Actual answer: {answer_actual}'
    )

    # Calculate accuracy and F1-score
    accuracy = accuracy_score([answer_actual], [predicted_answer])
    f1 = f1_score([answer_actual], [predicted_answer], average='macro')

    print(f'- Accuracy: {accuracy}\n- F1-score: {f1}')

    return question, predicted_answer, accuracy, f1

In [63]:
for question, context, answer_actual in zip(question[:10], context[:10], answer_actual[:10]):
    run_prediction(model, tokenizer, context, question, answer_actual)
    print('---')

- Question: Paris đạt được thành quả gì sau khoảng 4 thế kỷ tính từ ngày Cách mạng Pháp diễn ra?
-Context: Paris nằm ở điểm gặp nhau của các hành trình thương mại đường bộ và đường sông, và là trung tâm của một vùng nông nghiệp giàu có. Vào thế kỷ 10, Paris đã là một trong những thành phố chính của Pháp cùng các cung điện hoàng gia, các tu viện và nhà thờ. Từ thế kỷ 12, Paris trở thành một trong những trung tâm của châu Âu về giáo dục và nghệ thuật. Thế kỷ 14, Paris là thành phố quan trọng bậc nhất của Cơ Đốc giáo và trong các thế kỷ 16, 17, đây là nơi diễn ra Cách mạng Pháp cùng nhiều sự kiện lịch sử quan trọng của Pháp và châu Âu. Đến thế kỷ 19 và 20, thành phố trở thành một trong những trung tâm văn hóa của thế giới, thủ đô của nghệ thuật và giải trí.
- Predicted answer: nông nghiệp giàu có
- Actual answer: trở thành một trong những trung tâm văn hóa của thế giới, thủ đô của nghệ thuật và giải trí
- Accuracy: 0.0
- F1-score: 0.0
---
- Question: Kinh tế xung quanh kinh đô ánh sáng mạ